In [14]:
# Read data from Silver

silver_df = spark.read.table("Merged_Data_Silver")


StatementMeta(, 82b0e985-dd47-4177-93bc-ba5de8c351e1, 16, Finished, Available, Finished)

In [15]:
# Change Column names to 

silver_df = silver_df.toDF(*[col.lower() for col in silver_df.columns])
display(silver_df)

StatementMeta(, 82b0e985-dd47-4177-93bc-ba5de8c351e1, 17, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 69ec42eb-e334-47d5-8f79-0884888a5b99)

In [16]:
from pyspark.sql.functions import to_date

silver_df = silver_df.withColumn("sales_id", silver_df["sales_id"].cast("int")) \
       .withColumn("sales_date", to_date("sales_date", "yyyy-MM-dd")) \
       .withColumn("product_name", silver_df["product_name"].cast("string")) \
       .withColumn("category", silver_df["category"].cast("string")) \
       .withColumn("sub_category", silver_df["sub_category"].cast("string")) \
       .withColumn("customer_name", silver_df["customer_name"].cast("string")) \
       .withColumn("region", silver_df["region"].cast("string")) \
       .withColumn("customer_type", silver_df["customer_type"].cast("string")) \
       .withColumn("quantity_sold", silver_df["quantity_sold"].cast("int")) \
       .withColumn("unit_price", silver_df["unit_price"].cast("double")) \
       .withColumn("discount", silver_df["discount"].cast("double")) \
       .withColumn("total_amount", silver_df["total_amount"].cast("double"))

StatementMeta(, 82b0e985-dd47-4177-93bc-ba5de8c351e1, 18, Finished, Available, Finished)

In [17]:
silver_df.printSchema()

StatementMeta(, 82b0e985-dd47-4177-93bc-ba5de8c351e1, 19, Finished, Available, Finished)

root
 |-- sales_id: integer (nullable = true)
 |-- sales_date: date (nullable = true)
 |-- product_name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sub_category: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- region: string (nullable = true)
 |-- customer_type: string (nullable = true)
 |-- quantity_sold: integer (nullable = true)
 |-- unit_price: double (nullable = true)
 |-- discount: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- last_updated_date: timestamp (nullable = true)



In [18]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Step 1: dim_date
dim_date = silver_df.select("sales_date").distinct() \
    .withColumn("date_id", row_number().over(Window.orderBy("sales_date"))) \
    .withColumn("day", dayofmonth("sales_date")) \
    .withColumn("month", month("sales_date")) \
    .withColumn("month_name", date_format("sales_date", "MMMM")) \
    .withColumn("quarter", quarter("sales_date")) \
    .withColumn("year", year("sales_date")) \
    .withColumn("week_of_year", weekofyear("sales_date"))

# Step 2: dim_product
dim_product = silver_df.select("product_name", "category", "sub_category").distinct() \
    .withColumn("product_id", monotonically_increasing_id())

# Step 3: dim_customer
dim_customer = silver_df.select("customer_name", "customer_type").distinct() \
    .withColumn("customer_id", monotonically_increasing_id())

# Optional: dim_region (if region is simple)
dim_region = silver_df.select("region").distinct() \
    .withColumn("region_id", monotonically_increasing_id())

# Step 4: Create lookup versions of dimension tables for joins
product_lookup = dim_product.select("product_id", "product_name", "category", "sub_category")
customer_lookup = dim_customer.select("customer_id", "customer_name", "customer_type")
date_lookup = dim_date.select("date_id", "sales_date")
region_lookup = dim_region.select("region_id", "region")

# Step 5: fact_sales
fact_sales = silver_df \
    .join(product_lookup, ["product_name", "category", "sub_category"], "left") \
    .join(customer_lookup, ["customer_name", "customer_type"], "left") \
    .join(date_lookup, ["sales_date"], "left") \
    .join(region_lookup, ["region"], "left") \
    .select(
        col("sales_id"),
        col("date_id"),
        col("product_id"),
        col("customer_id"),
        col("region_id"),
        col("quantity_sold"),
        col("unit_price"),
        col("discount"),
        col("total_amount"),
        col("last_updated_date")
    )


StatementMeta(, 82b0e985-dd47-4177-93bc-ba5de8c351e1, 20, Finished, Available, Finished)

In [19]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

StatementMeta(, 82b0e985-dd47-4177-93bc-ba5de8c351e1, 21, Finished, Available, Finished)

In [20]:
dim_product.write.mode("overwrite").format("csv").save("Files/Gold/dim_product")
dim_customer.write.mode("overwrite").format("csv").save("Files/Gold/dim_customer")
dim_region.write.mode("overwrite").format("csv").save("Files/Gold/dim_region")
dim_date.write.mode("overwrite").format("csv").save("Files/Gold/dim_date")
fact_sales.write.mode("overwrite").format("csv").save("Files/Gold/fact_sales")

StatementMeta(, 82b0e985-dd47-4177-93bc-ba5de8c351e1, 22, Finished, Available, Finished)

In [21]:
dim_product.write.mode("overwrite").format("delta").saveAsTable("dim_product")
dim_customer.write.mode("overwrite").format("delta").saveAsTable("dim_customer")
dim_region.write.mode("overwrite").format("delta").saveAsTable("dim_region")
dim_date.write.mode("overwrite").format("delta").saveAsTable("dim_date")
fact_sales.write.mode("overwrite").format("delta").saveAsTable("fact_sales")

StatementMeta(, 82b0e985-dd47-4177-93bc-ba5de8c351e1, 23, Finished, Available, Finished)